In [19]:
#split data into 20 categories (10-90% percentile for positive and negative so 10x2) based on SMA 1 minute from past 20 days, 
#then within each SMA category, separate into quartiles based on cumulative volume in 1 minute
#calculate the mean and std of the edge in each case

import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
from sklearn import linear_model
import matplotlib.pyplot as plt

path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA'
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir(path)
file_list.sort()

In [20]:
file_list

['.DS_Store',
 '2019.01.02.csv',
 '2019.01.03.csv',
 '2019.01.04.csv',
 '2019.01.07.csv',
 '2019.01.08.csv',
 '2019.01.09.csv',
 '2019.01.10.csv',
 '2019.01.11.csv',
 '2019.01.14.csv',
 '2019.01.15.csv',
 '2019.01.16.csv',
 '2019.01.17.csv',
 '2019.01.18.csv',
 '2019.01.21.csv',
 '2019.01.22.csv',
 '2019.01.23.csv',
 '2019.01.24.csv',
 '2019.01.25.csv',
 '2019.01.28.csv',
 '2019.01.29.csv',
 '2019.01.30.csv',
 '2019.01.31.csv',
 '2019.02.01.csv',
 '2019.02.11.csv',
 '2019.02.12.csv',
 '2019.02.13.csv',
 '2019.02.14.csv',
 '2019.02.15.csv',
 '2019.02.18.csv',
 '2019.02.19.csv',
 '2019.02.20.csv',
 '2019.02.21.csv',
 '2019.02.22.csv',
 '2019.02.25.csv',
 '2019.02.26.csv',
 '2019.02.27.csv',
 '2019.02.28.csv',
 '2019.03.01.csv',
 '2019.03.04.csv',
 '2019.03.05.csv',
 '2019.03.06.csv',
 '2019.03.07.csv',
 '2019.03.08.csv',
 '2019.03.11.csv',
 '2019.03.12.csv',
 '2019.03.13.csv',
 '2019.03.14.csv',
 '2019.03.15.csv',
 '2019.03.18.csv',
 '2019.03.19.csv',
 '2019.03.20.csv',
 '2019.03.21.csv'

In [21]:
class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
class categorise_simple():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        self.threshold.append(np.percentile(array,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)
        return np.asarray(temp)    
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(11)
                elif i<self.threshold[1][1]:
                    temp.append(12)
                elif i<self.threshold[1][2]:
                    temp.append(13)
                elif i<self.threshold[1][3]:
                    temp.append(14)
                elif i<self.threshold[1][4]:
                    temp.append(15)
                elif i<self.threshold[1][5]:
                    temp.append(16)
                elif i<self.threshold[1][6]:
                    temp.append(17)
                elif i<self.threshold[1][7]:
                    temp.append(18)
                elif i<self.threshold[1][8]:
                    temp.append(19)                    
                else:
                    temp.append(20)
            if i<0:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)
                elif i<self.threshold[0][8]:
                    temp.append(9)                    
                else:
                    temp.append(10)
        return np.asarray(temp)    
class cross():
    def __init__(self):
        self.time_last_cross=0
        self.current_sign=True
        self.last_time=datetime(1900, 1, 1, 8, 59)
    def get_time(self,time,price):
        if (time-self.last_time)>timedelta(minutes=1):
            self.last_time=time
            self.time_last_cross=time
            return 0
        self.last_time=time
        if (price>0) and self.current_sign : #if price positive and current trend is also positive
            return (time-self.time_last_cross).total_seconds()
        elif (price<0) and (not self.current_sign): #if price negative and current trend is negative
            return (time-self.time_last_cross).total_seconds()
        else: #if price positive, trend negative or price negative, trend positive
            self.time_last_cross=time
            self.current_sign=(price>0)
            return 0

In [22]:
def calc_smart_price(dataset):
    data=dataset[:]
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_present_vol(dataset):
    data=dataset[:]
    data['current_vol']=data.loc[:,'Volume'].diff().fillna(0)/2
    return data

def calc_future_price(dataset,time_ahead=30,time_index=44, price_col=-2):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,time_index]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),time_index]):
            j+=1
        if (i+j)<length:
            future_price.append(data[(i+j),price_col]) 
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset,future_col=-1,current_col=-3):
    data=dataset.copy()
    temp=data[:,future_col]-data[:,current_col]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset,time_index=44):
    data=dataset[:]
    index=data[:,time_index]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,time_index]=new_index
    return data

def calc_sma_fast(dataset,duration=1,time_index=44,price_col=-4): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,price_col]) #smart price column is -4
    for i in range(len(data)):
        last_time=data[i,time_index]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,time_index]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)

    sma_values=np.asarray(sma_values)
    sma_values=data[:,price_col]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)  

def calc_past_vol(dataset,duration=1,time_index=44,vol_col=-4): #
    data=dataset[:].copy()
    vol_values=[] 
    vol_sum=np.cumsum(data[:,vol_col])
    for i in range(len(data)):
        last_time=data[i,time_index]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,time_index]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) 

def last_cross(dataset,time_index=44,price_col=-3):
    data=dataset[:]
    last_cross=cross()
    timings=[]
    for i in range(len(data)):
        timings.append(last_cross.get_time(data[i,time_index],data[i,price_col]))
    timings=np.asarray(timings)
    timings=np.expand_dims(timings,axis=1)
    return np.concatenate((data,timings),axis=1)
def process(dataset,sma_duration=1,vol_duration=1,time_index=44):
    data=dataset[:]
    data=calc_smart_price(data).values #new
    data=set_index(data,time_index=time_index) #no change
    data=calc_future_price(data,time_index=time_index,price_col=-1) #new
    data=calc_edge(data,future_col=-1,current_col=-2) #new
    data=calc_sma_fast(data,duration=sma_duration,time_index=time_index,price_col=-3) #new
    data=last_cross(data,time_index=time_index,price_col=-1) #new
    return data

In [64]:
#processing raw data to get technicals
df_list=[]
name_list=[]
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
for file in file_list: #read all files and add them to file_list
    if file[-3:]=='csv': #check if file is a CSV
        name_list.append(file)
        df_list.append(process(pd.read_csv(path+file),sma_duration=5))
        print(file,'read')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2019.01.02.csv read
2019.01.03.csv read
2019.01.04.csv read
2019.01.07.csv read
2019.01.08.csv read
2019.01.09.csv read
2019.01.10.csv read
2019.01.11.csv read
2019.01.14.csv read
2019.01.15.csv read
2019.01.16.csv read
2019.01.17.csv read
2019.01.18.csv read
2019.01.21.csv read
2019.01.22.csv read
2019.01.23.csv read
2019.01.24.csv read
2019.01.25.csv read
2019.01.28.csv read
2019.01.29.csv read
2019.01.30.csv read
2019.01.31.csv read
2019.02.01.csv read
2019.02.11.csv read
2019.02.12.csv read
2019.02.13.csv read
2019.02.14.csv read
2019.02.15.csv read
2019.02.18.csv read
2019.02.19.csv read
2019.02.20.csv read
2019.02.21.csv read
2019.02.22.csv read
2019.02.25.csv read
2019.02.26.csv read
2019.02.27.csv read
2019.02.28.csv read
2019.03.01.csv read
2019.03.04.csv read
2019.03.05.csv read
2019.03.06.csv read
2019.03.07.csv read
2019.03.08.csv read
2019.03.11.csv read
2019.03.12.csv read
2019.03.13.csv read
2019.03.14.csv read
2019.03.15.csv read
2019.03.18.csv read
2019.03.19.csv read


In [65]:
#calculating results
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'

#create a list to hold all the data
data_list=[]
#creating a list for each category
for i in range(20):
    data_list.append([])
    #creating a list for each quartile
    for _ in range(4):
        data_list[i].append([])

final_df=pd.DataFrame()    

for i in range(len(df_list)): #for each 20 day rolling window
    if i<19: #skip first 19 days
        continue
    print(name_list[i])

    #get -19 day
    sma=df_list[i-19][:,-2].copy() #column for SMA
    time_cross=df_list[i-19][:,-1].copy()
    
    #get -18 to 0 day (19 days in total)
    for k in range((i-18),i+1): #get 20 day moving averages
        sma=np.concatenate((sma,df_list[k][:,-2].copy()))
        time_cross=np.concatenate((time_cross,df_list[k][:,-1].copy()))
        
    cat_sma=categorise_10()
    cat_sma.fit(sma) #calculate quartile thresholds for past 20 days
    
    #get categories for past 20 days
    cat_sma_20=(cat_sma.return_quartile(sma))   

    #get x,y for regression
    x_today=df_list[i][:,-2].copy().astype(float) #column for SMA     
    y_today=df_list[i][:,-3].copy().astype(float) #column for edge
    time_cross_today=df_list[i][:,-1].copy() #column for time since last cross
    
    #removing all NA
    isnum=(~np.isnan(x_today)) & (~np.isnan(y_today))
    #get categories of today's sma  
    cat_x_today=cat_sma.return_quartile(x_today)
    
    #for each category
    for cat in range(1,21):
        #check past 20 days quartiles and filter time_cross by category
        time_cross_fit=time_cross[cat_sma_20==cat].copy()

        #simple quartile categorisation
        cat_time=categorise_simple()
        cat_time.fit(time_cross_fit)
    
        #today's sma filter
        sma_filter_today=(cat_x_today==cat)
        #today's time categories
        cat_time_today=cat_time.return_quartile(time_cross_today)

        #for each quartile
        for quartile in [1,2,3,4]:

            filtered= (isnum & sma_filter_today) #filtering NA and sma quartile
            filtered= filtered & (cat_time_today==quartile) #filtering volume
            new_y=y_today[filtered].copy()

            if (len(new_y)!=0):

                data_list[(cat-1)][(quartile-1)].append(new_y)
reg_result={}                 
for cat in range(20):
    for quartile in range(4):
        all_data=data_list[cat][quartile][0]
        for i in range(1,(len(data_list[cat][quartile]))):
            all_data=np.concatenate((all_data,data_list[cat][quartile][i]))
        reg_result['category']=cat+1
        reg_result['quartile']=quartile+1
        reg_result['mean']=np.mean(all_data)
        reg_result['std']=np.std(all_data)
        reg_result['num obs']=len(all_data)
        final_df=final_df.append(reg_result,ignore_index=True)  
             
temp=df_path+'result_10split_time_cross_5min_sma_weightedmean.csv'
final_df.to_csv(temp)
print('done',temp)

2019.01.29.csv
2019.01.30.csv
2019.01.31.csv
2019.02.01.csv
2019.02.11.csv
2019.02.12.csv
2019.02.13.csv
2019.02.14.csv
2019.02.15.csv
2019.02.18.csv
2019.02.19.csv
2019.02.20.csv
2019.02.21.csv
2019.02.22.csv
2019.02.25.csv
2019.02.26.csv
2019.02.27.csv
2019.02.28.csv
2019.03.01.csv
2019.03.04.csv
2019.03.05.csv
2019.03.06.csv
2019.03.07.csv
2019.03.08.csv
2019.03.11.csv
2019.03.12.csv
2019.03.13.csv
2019.03.14.csv
2019.03.15.csv
2019.03.18.csv
2019.03.19.csv
2019.03.20.csv
2019.03.21.csv
2019.03.22.csv
2019.03.25.csv
2019.03.26.csv
2019.03.27.csv
2019.03.28.csv
2019.03.29.csv
2019.04.01.csv
2019.04.02.csv
2019.04.03.csv
2019.04.04.csv
2019.04.08.csv
2019.04.09.csv
2019.04.10.csv
2019.04.11.csv
2019.04.12.csv
2019.04.15.csv
2019.04.16.csv
2019.04.17.csv
2019.04.18.csv
2019.04.19.csv
2019.04.22.csv
2019.04.23.csv
2019.04.24.csv
2019.04.25.csv
2019.04.26.csv
2019.04.29.csv
2019.04.30.csv
2019.05.06.csv
2019.05.07.csv
2019.05.08.csv
2019.05.09.csv
done /Users/hudsonyeo/Desktop/Python/leo

In [67]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_10split_time_cross_5min_sma_weightedmean.csv')
mean=np.reshape(data.loc[:,'mean'].values,(20,4))
std=np.reshape(data.loc[:,'std'].values,(20,4))
num=np.reshape(data.loc[:,'num obs'].values,(20,4))
print('mean')
print(mean)
print('std')
print(std)
print('num obs')
print(num)

mean
[[-0.19267159  0.12752573  0.02202854  0.31957321]
 [-0.25649618  0.12531803  0.02666349 -0.0112897 ]
 [-0.1846334   0.03117279 -0.12877952  0.09156817]
 [-0.24075646 -0.08636318 -0.05001419 -0.22235719]
 [-0.27248255 -0.26193325 -0.03783178 -0.14843497]
 [-0.30906995 -0.23135513 -0.06245333 -0.01490423]
 [-0.21057097 -0.13290848 -0.1945946   0.0344212 ]
 [-0.2268676  -0.22584797 -0.03771642 -0.01787789]
 [-0.22338218 -0.31376897 -0.01367657  0.12721989]
 [-0.16607222 -0.22488814 -0.09246023  0.03626679]
 [ 0.1364673   0.11498446  0.0645534  -0.37262921]
 [ 0.31453199  0.18066004 -0.19912584 -0.18409987]
 [ 0.32758098  0.20738517  0.00116818 -0.0871107 ]
 [ 0.37557096  0.13040826 -0.00643352 -0.10886664]
 [ 0.34922263  0.12666693  0.16250199 -0.12311086]
 [ 0.37120529  0.16834032  0.0403432  -0.10191035]
 [ 0.45741924  0.18467509  0.1779754  -0.09037451]
 [ 0.2875211   0.32216369  0.14377845 -0.1403605 ]
 [ 0.38955228  0.25750677 -0.01039145 -0.08760104]
 [ 0.58069136 -0.1557486  